# FireCrawl example

- [firecrawl](https://firecrawl.dev) 테스트
- DOM 구조를 몰라도 의미적으로 필요한 정보를 가져올 수 있다.

## Install Deps

In [2]:
!pip -q install firecrawl-py -U

## Setup

In [3]:
import os
from getpass import getpass

from pydantic import BaseModel, Field
from firecrawl import FirecrawlApp

In [4]:
FIRE_CRAWL_API_KEY = os.environ.get("FIRE_CRAWL_API_KEY")
if not FIRE_CRAWL_API_KEY:
    FIRE_CRAWL_API_KEY = getpass("FIRE_CRAWL_API_KEY: ")

FIRE_CRAWL_API_KEY:  ········


In [5]:
app = FirecrawlApp(api_key=FIRE_CRAWL_API_KEY)

## Usage

In [6]:
url = "https://lol.inven.co.kr/dataninfo/champion/manualTool.php?confirm=2"
url

'https://lol.inven.co.kr/dataninfo/champion/manualTool.php?confirm=2'

In [9]:
class Tactic(BaseModel):
    link: str
    title: str
    champion: str
    season: list[str]
    published_at: str

class ExtractSchema(BaseModel):
    tactics: list[Tactic]

In [10]:
ExtractSchema.model_json_schema()

{'$defs': {'Tactic': {'properties': {'link': {'title': 'Link',
     'type': 'string'},
    'title': {'title': 'Title', 'type': 'string'},
    'champion': {'title': 'Champion', 'type': 'string'},
    'season': {'items': {'type': 'string'},
     'title': 'Season',
     'type': 'array'},
    'published_at': {'title': 'Published At', 'type': 'string'}},
   'required': ['link', 'title', 'champion', 'season', 'published_at'],
   'title': 'Tactic',
   'type': 'object'}},
 'properties': {'tactics': {'items': {'$ref': '#/$defs/Tactic'},
   'title': 'Tactics',
   'type': 'array'}},
 'required': ['tactics'],
 'title': 'ExtractSchema',
 'type': 'object'}

In [14]:
%%time

data = app.scrape_url(url, {
    'formats': ['extract'],
    'extract': {
        'prompt': 'Extract the user published tactics from the table tag in the page.',
        'schema': ExtractSchema.model_json_schema(),
    }
})
print(data.keys())

dict_keys(['metadata', 'extract'])
CPU times: user 14.5 ms, sys: 7.33 ms, total: 21.9 ms
Wall time: 15.2 s


In [15]:
tactics = data['extract']['tactics']
print(len(tactics))
tactics

5


[{'link': 'manualToolView.php?idx=148111',
  'title': '미드 카서스',
  'champion': '카서스',
  'season': ['시즌14'],
  'published_at': '09-22'},
 {'link': 'manualToolView.php?idx=148105',
  'title': '힐 집중 강화 니달리 서폿 공략(업데이트)',
  'champion': '니달리',
  'season': ['시즌14'],
  'published_at': '09-22'},
 {'link': 'manualToolView.php?idx=109083',
  'title': 'SeasonⅤ Ezreal - The Prodigal Explorer',
  'champion': '이즈리얼',
  'season': ['시즌Ⅴ'],
  'published_at': '09-22'},
 {'link': 'manualToolView.php?idx=148110',
  'title': '앙?',
  'champion': '케인',
  'season': ['시즌14'],
  'published_at': '09-22'},
 {'link': 'manualToolView.php?idx=148109',
  'title': '브라이어',
  'champion': '브라이어',
  'season': ['시즌14'],
  'published_at': '09-22'}]